# Lab 7: GraphQL

*Due 5 PM, Tuesday April 9, 2019*

The goal of this lab is to learn GraphQL by gradually building up an API from the ground up. We will recreate and extend the University App from previous labs today. 

## Lab Problem 1: Create a GraphQL Sandbox

1. Go to [Codesandbox.io](https://codesandbox.io) and create an Apollo Sandbox. Apollo is a web server written in JavaScript (NodeJS), specially designed for implemented GraphQL endpoints. 

1. To create an Apollo sandbox, press the plus (+) button on the top right of the window, then select "Server Templates" and "Apollo" from the grid.

1. You now have a minimal Apollo sandbox, a template that you will extend in the following exercises.

Study the structure of the code. You should see three main sections. 

The first part `typedefs` defines the GraphQL *schema*, the interface of your API. In GraphQL the schema consists of four pieces: a root *query* section where you define queries on the API, a *mutation* section where you define operations that change or update state on the server, one or more *types* which declare the arguments and return values passed and returned by queries and mutations. Finally, a GraphQL schema can have a *subscription* section that allows clients to subscribe to changes on the server.

## Lab Problem 2: Explore the GraphQL API

GraphQL is self-documenting in that the schema definition can be examined by clients and browsed. The browser window in the codesandbox should have a green "Schema" tab on the right. Click on it. Normally, a schema for a real app will have many queries, mutations, and types. Our stub currently has just one query: `hello`, which takes no arguments and returns a `String`.

At this point you should probably copy the browser URL from the browser tab and paste it into a new browser window. This will give you more room to work.

You are now ready to issue your first query:

```graphql
query {
  hello
}
```

Type the above and hit the Play button. You should see the results on the right side of the screen.

Notice how query editor helps with completion as you type. This is because the editor is aware of the schema.

Examine the `resolvers` section in `index.js`. Notice the `hello` entry. This simply maps a query as defined in the schema to a Javascript function. The stub implementation just returns "Hello world!"

## Lab Problem 3: Pass a Parameter to the Hello Query

We're going to pass a `name` parameter to the `hello` query, which will return `Hello ${name}!` instead of "Hello World!". 

1. Modify the schema for `hello` to add a `name` parameter like this:
 ```javascript
const typeDefs = gql`
  type Query {
    hello(name: String!): String
  }
`; 
 ```
 The above specifies that `name` will be a GraphQL primitive String. The exclamation after the type tells GraphQL that the parameter is required.
1. Modify the resolver for `hello` to return the name:
 ```javascript
    hello: (root, args, context) => `Hello ${args.name}!`
 ```
 
Now reload the schema browser window so that it has the latest schema definition and re-submit the same `hello` query as before. You should get an error on the right.

Edit the query to specify the name and re-run the query: 
```graphql
query {
  hello(name: "Your name")
}
```

## Lab Problem 4: Add Your First User-defined Type

Add a `User` type to the schema:

```graphql
enum Role {
    Admin
    Student
    Faculty
  }

type User {
  id: ID!
  name: String!
  email: String!
  role: Role!
}
```

We've actually created two types. The first is a `Role`, an enumeration of valid user roles and a `User` type which includes the role as one of it's fields.

Reload the schema browser and click on the green Schema tab.

## Lab Problem 5: Add 'Users' Query

We're now going to add a 'users' query to the schema by modifying the `query` section.

```graphql
type Query {
  hello(name: String!): String
  users: [User]
}
```

Notice the `[Users]` notation, which means that the `users` query will return a list of Users.

At this point you can reload the schema browser and explory the `users` query. We haven't yet implemented the query. So running it, will yield an error.

To implement the query, we'll create a class called Users:

```javascript
class Users {
  constructor() {
    this.nextID = 2;
    this.users = [
      { id: 0, name: "zero", email: "zero@example.com", role: "Admin" },
      { id: 1, name: "one", email: "one@example.com", role: "Student" },
      { id: 2, name: "prof", email: "admin@example.com", role: "Faculty" }
    ];
  }

  getUsers() {
    return this.users;
  }
}
```

Now add the following above the `resolvers` section to create a new users object:

```javascript
const users = new Users();
```

Finally, add a resolver for `users`:

```javascript
    users: (root, args, context) => users.getUsers()
```

Use the browser query the server for `users`. Your first attempt will likely be unsuccessful because GraphQL requires you to say exactly which subset of user attributes you want to retrieve.

## Lab Problem 6: Create a User with A GraphQL Mutation

We're now ready for our first mutation.

Add a `Mutation` section to the schema:
```graphql
type Mutation {
    createUser(name: String!, email: String!, role: Role!): User
}
```

Also add a `Mutation` section to the resolver, which should look like this:

```javascript
const resolvers = {
  Query: {
    hello: (root, args, context) => `Hello ${args.name}!`,
    users: (root, args, context) => users.getUsers()
  },
  Mutation: {
    createUser: (_, { user }, context) => users.create(user)
  }
};
```

Implement a `create` method in the `Users` class. It will take an object with three fields (name, email, role), create a new `User` object, push it onto the `users` list, and return the new `user` object.

Relead the schema browser. You should see `createUser` under the `MUTATIONS` section.

Write a mutation to create a new user and then retrieve the users with a query. You can use separate tabs for mutations and queries.

## Lab Problem 7: Create Student, Faculty, and Admin Types

Our app manages three types of users: Admins, Students, and Faculty. We want to track them separately as distinct types with distinct operations but yet use a single database table (`Users`) for all three. (The role attribute in the database will distinguish between the three.)

To do this, we'll create a special GraphQL type called an 'interface', whose definition can be shared by concrete types such as `Student`, `Faculty`, and `Admin`.

Replace the `User` type in the schema with the following:

```graphql
interface User {
  id: ID!
  name: String!
  email: String!
  role: Role!
  }

type Student implements User {
  id: ID!
  name: String!
  email: String!
  role: Role!
  #    courses: [Course]
  gpa: Float!
}

type Faculty implements User {
  id: ID!
  name: String!
  email: String!
  role: Role!
  #    courses: [Course]
}

type Admin implements User {
  id: ID!
  name: String!
  email: String!
  role: Role!
}
```

You now have an abstract `User` type and three concrete types that implement the `User` interface. 

Saving the file will generate a warning because the resolver needs help in determining how to distinguish between the three concrete types at runtime. To do this, add a resolver for `User`:

```javascript
User: {
  __resolveType: (user, context, info) => user.role
}
```

## Lab Problem 8: Add Queries for Faculty and Students

Add two queries one for `faculty` and one for `students` to the schema. Implement methods in the `Users` class to filter on the respective role. Finally, add resolvers for these queries.

Try out the queries from the browser.

## Lab Problem 9: Add Courses, Assignments, and Assignment Grades

We're now ready to add types for courses, assignments, and grades:

```graphql
type Course {
  id: ID!
  name: String!
  professor: Faculty
  students: [Student]
  assignments: [Assignment]
}

type Assignment {
  id: ID!
  name: String!
  course: Course!
  grades: [AssignmentGrade]
}

type AssignmentGrade {
  id: ID!
  assignment: Assignment
  student: User
  grade: String!
}
```

# Homework
In the homework, you will implement queries and mutations for courses and assignments.

## Homework Problem 1: Retrieve a Single Faculty or Student by email or ID

*20 Points*

Add and implement the following two queries:

```graphql
    student(email: String!, id: ID): Student
    faculty(email: String!, id: ID): Faculty
```

## Homework Problem 2: Course Mutations

*40 Points*

Create a `Courses` class, just like the `Users` class. Implement the following mutations:

```graphql
  # 10 Points: Create a course, with given name and faculty ID
  # the course name should be unique
  createCourse(name: String!, facultyID: ID!): Course
  
  # 10 Points: Delete a course
  deleteCourse(courseID: ID!): Course
  
  # 10 Points: Add a student to a course. Do nothing if the student
  # is already enrolled
  addCourseStudent(courseID: ID!, studentID: ID!): Course
  
  # 10 Points: Remove a student from a course
  deleteCourseStudent(courseID: ID!, studentID: ID!): Course
```

## Homework Problem 3: Course Queries

*10 Points*

Add a top-level query called `courses` to retrieve all the courses. 

## Homework Problem 4: List Courses for Students

*10 Points*

Add a field `courses: [Course]` to the `Student` and `Faculty` types.
Retrieve courses for taken by individual students and given by each faculty.

## Homework Problem 5: Assignments

*30 points*

Implement an `Assignments` class along with the following mutations:
```
  # 10 Points: Create an assignment for a given course
  createAssignment(courseID: ID!, name: String!): Assignment
  
  # 10 Points: Delete an assignment by ID
  deleteAssignment(assignmentID: ID!) : Assignment
  
  # 10 Points: Set a grade for a student by specifying the
  # studentID and grade
  createAssignmentGrade(
    assignmentID: ID!
    studentID: ID!
    grade: Float!
  ): AssignmentGrade
```



## Homework Submission

Please submit a link to your forked and modified codesandbox app to Gradescope. Precise instructions to follow.